In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

import time

import pyautogui as pya

import re

import pandas as pd
import os

In [166]:
temp_text=[]
temp_chaps=[]
book_idx=1
target="New Revised Standard Version"

In [152]:
opts=webdriver.ChromeOptions()
opts.add_argument('--incognito')
driver=webdriver.Chrome(options=opts)
site='https://www.bible.com/it/bible/130/GEN.1.TOJB2011'
driver.get(site)


In [167]:
while 1==1:
    text_body=driver.find_element(By.CLASS_NAME,'yv-bible-text ')
    error_patt_inside_last='(\d+\s?\D+)$'
    error_patt_inside='(\d+\s\D+)(\d+\D+)'
    error_patt_start='(^\d+)(\D+)(\d+\D+)' # 1 2 for 12
    error_patt_start2='(^\d+)(\‘)(\D+)' # 4'verse for 4 verse
    
    title_temp=text_body.find_element(By.TAG_NAME,'h1').text
    title=' '.join(title_temp.split(' ')[:-1]).title()
    chapter=title_temp.split(' ')[-1]
    text='\n'.join(text_body.text.split('\n')[1:]).replace('\n','')
    
    text=re.sub("(\d+)(\w)", r"\n\1 \2",text,flags=re.I).split('\n')
    
    text=[x for x in text if x !='']
    text_check=[[z for y in re.findall(error_patt_inside,x) for z in y] if re.findall(error_patt_inside,x)!=[] else [text[i]] for i,x in enumerate(text)]#missing last verse for clustered verses
    #check last for non-[] els
    text_check2=[[z for z in re.findall(error_patt_inside_last,text[i])] if x!=[] else [] for i,x in enumerate(text_check)]
    #join text_checks all+last
    try:
        [x.append(text_check2[i]) if text_check[i]!=[''] or text_check[i]!=[] else text[i] for i,x in enumerate(text_check)]
        text=[chapter+'__'+re.sub(error_patt_start2,r'\1 \2\3', re.sub(error_patt_start,r'\1\3 ',str(y))) for x in text_check for y in x]
    except:
        text=[chapter+'__'+x for x in text]
    [temp_text.append(re.sub('(^\d+)(\D)',r'\1 \2',x)) for x in text if re.sub('(^\d+)(\D)',r'\1 \2',x) not in temp_text ]
    [temp_chaps.append(chapter) for x in text]
    try:
        next_page=driver.find_element(By.XPATH,'//div[@class="next-arrow"]//a').get_attribute('href')
    except:
        print('end')
        t_chaps=[x.split('_')[0] for x in temp_text]
        t_verses=[' '.join(x.split('__')[1:]).strip() for x in temp_text]
        t_verses_idx=[x.split(' ')[0] for x in t_verses]
        t_verses_idx=[re.sub('\D','',x) for x in t_verses]
        df=pd.DataFrame()
        df['Edition']=[target for x in t_verses ]
        df['Book']=[title for x in t_verses]
        df['Chapter']=[x for x in t_chaps]
        df['Verse_idx']=t_verses_idx
        df['Verse_']=[' '.join(x.split(' ')[1:]) for x in t_verses]
        df.drop_duplicates(subset=['Book','Chapter','Verse_idx'],inplace=True)
        df.reset_index(drop=True,inplace=True)
        df['Verse']=[df['Verse_idx'][i]+' '+df['Verse_'][i] for i,x in enumerate(df['Verse_idx'])]
        df.drop(columns=['Verse_idx','Verse_'],axis=1,inplace=True)

        df.to_csv(f'./csv bibles/{book_idx}-{target};{title}.csv')
        print(title, ' df created')
        book_idx+=1        
        temp_text=[]
        temp_chaps=[]
        print(title, ' df created')     
        break #exit on last, if there's no next page
    next_url_book_fragment=next_page.split('/')[-1].split('.')[0]
    curr_url_book_fragment=driver.current_url.split('/')[-1].split('.')[0]
    #[temp_text.append(re.sub('(^\d+)(\D)',r'\1 \2',x)) for x in text]
    #[temp_chaps.append(chapter) for x in text]
    if next_url_book_fragment!=curr_url_book_fragment:
        t_chaps=[x.split('_')[0] for x in temp_text]
        t_verses=[' '.join(x.split('__')[1:]).strip() for x in temp_text]
        t_verses_idx=[x.split(' ')[0] for x in t_verses]
        t_verses_idx=[re.sub('\D','',x) for x in t_verses]
        df=pd.DataFrame()
        df['Edition']=[target for x in t_verses ]
        df['Book']=[title for x in t_verses]
        df['Chapter']=[x for x in t_chaps]
        df['Verse_idx']=[x.split(' ')[0] for x in t_verses_idx]
        df['Verse_']=[' '.join(x.split(' ')[1:]) for x in t_verses]
        df.drop_duplicates(subset=['Book','Chapter','Verse_idx'],inplace=True)
        df.reset_index(drop=True,inplace=True)
        df['Verse']=[df['Verse_idx'][i]+' '+df['Verse_'][i] for i,x in enumerate(df['Verse_idx'])]
        df.drop(columns=['Verse_idx','Verse_'],axis=1,inplace=True)

        print('correcting')


        df.to_csv(f'./csv bibles/{book_idx}-{target};{title}.csv')
        print(title, ' df created')
        book_idx+=1        
        temp_text=[]
        temp_chaps=[]
    #print(text_check_final)
    #print(text_body, temp_text)
    driver.get(next_page)
    time.sleep(4)
    


correcting
Genesis  df created
correcting
Exodus  df created
correcting
Leviticus  df created
correcting
Numbers  df created
correcting
Deuteronomy  df created
correcting
Joshua  df created
correcting
Judges  df created
correcting
Ruth  df created
correcting
1 Samuel  df created
correcting
2 Samuel  df created
correcting
1 Kings  df created
correcting
2 Kings  df created
correcting
1 Chronicles  df created
correcting
2 Chronicles  df created
correcting
Ezra  df created
correcting
Nehemiah  df created
correcting
Esther  df created
correcting
Job  df created
correcting
Psalms  df created
correcting
Proverbs  df created
correcting
Ecclesiastes  df created
correcting
Song Of Solomon  df created
correcting
Isaiah  df created
correcting
Jeremiah  df created
correcting
Lamentations  df created
correcting
Ezekiel  df created
correcting
Daniel  df created
correcting
Hosea  df created
correcting
Joel  df created
correcting
Amos  df created
correcting
Obadiah  df created
correcting
Jonah  df crea

In [43]:
temp_text

['1 __PARASHAS BERESHIS',
 '1 __[]',
 '1 __1 In the beginning Elohim bara hashomayim (the heavens) and haaretz (the earth).',
 "1 __['1 In the beginning Elohim bara hashomayim (the heavens) and haaretz (the earth).']",
 '1 __2 And the earth was tohu vavohu (without form, and void); and darkness was upon the face of the deep. And the Ruach Elohim was hovering upon the face of the waters.',
 "1 __['2 And the earth was tohu vavohu (without form, and void); and darkness was upon the face of the deep. And the Ruach Elohim was hovering upon the face of the waters.']",
 '1 __3 And Elohim said, Let there be light: and there was light.',
 "1 __['3 And Elohim said, Let there be light: and there was light.']",
 '1 __4 And Elohim saw the light, that it was tov (good); and Elohim divided the ohr (light) from the choshech (darkness).',
 "1 __['4 And Elohim saw the light, that it was tov (good); and Elohim divided the ohr (light) from the choshech (darkness).']",
 '1 __5 And Elohim called the light Y

#other site
books_btn= driver.find_elements(By.TAG_NAME,'input')[8:-8]

temp_chaps=[]
temp_verses=[]
temp_books=[]
target_ed='New World Translation'
book_idx=1
for i,each in enumerate(books_btn):
    books_btn=driver.find_elements(By.TAG_NAME,'input')[8:-8] 
    book=books_btn[i].get_attribute('value')
    books_btn[i].click()
    while 1==1:
        next_chap_btn=[x for x in driver.find_elements(By.TAG_NAME,'input') if x.get_attribute('value')=='Next Chapter >>'][0]
        verses=[x.text.split('\n') for x in driver.find_elements(By.TAG_NAME,'td') if re.match('^\d+:',x.text)][0]
        chaps=[x.split(':')[0] for x in verses]
        verses=[':'.join(x.split(':')[1:]) for x in verses]

        [temp_chaps.append(x) for x in chaps]
        [temp_verses.append(x) for x in verses]
        [temp_books.append(book) for x in verses]
        if next_chap_btn.get_attribute('disabled')=='true':
            df=pd.DataFrame()
            df['Edition']=[target_ed for x in temp_verses]
            df['Book']=[book for x in temp_verses]
            df['Chapter']=[x for x in temp_chaps]
            df['Verse']=[x for x in temp_verses]

            df.to_csv(f'./csv bibles/{book_idx}-{target_ed};{book}.csv')
            print(book, ' df created')
            book_idx+=1
            temp_verses=[]
            temp_chaps=[]
            temp_books=[]
            driver.get('https://nocr.net/hbm/english/engnwt/index.php')
            time.sleep(4)
            break
        else:
            next_chap_btn.click()
            time.sleep(4)
            
            
        
    


In [41]:
 [x for x in os.listdir(root_) if not (x.endswith('.csv'))]

NameError: name 'root_' is not defined

In [13]:
root_='./NEW_csv_folders'
#concat dfs
for folder in [x for x in os.listdir(root_) if not(x.endswith('.csv'))]:
    if folder=="Orthodox Jewish Bible": #remove after use and remove also extra tabs 
        print(folder)
        idxs=sorted([x for x in os.listdir(f'{root_}/{folder}/')],key=lambda x:int(x.split('-')[0]))
        df_l=[pd.read_csv(f'{root_}/{folder}/{x}') for x in idxs]
        concat_df=pd.concat(df_l,axis=0,ignore_index=True)
        unwanted_rows=concat_df[concat_df['Verse'].str.contains('^\D')].index
        concat_df.drop(unwanted_rows,axis=0,inplace=True)
        #concat_df['Chapter']=[int(x) for x in concat_df['Chapter']]
        concat_df.drop('Unnamed: 0',axis=1,inplace=True)
        concat_df.reset_index(drop=True,inplace=True)
        concat_df=concat_df.apply(lambda x: x.strip() if type(x)==str else x)
        concat_df['Edition']=[x.title() for x in concat_df['Edition']]
        concat_df=concat_df.drop_duplicates(ignore_index=True)
        concat_df.to_csv(f'{root_}/CONCAT_{folder}.csv',index=False,header=True)


        print(concat_df)

    

Orthodox Jewish Bible
                     Edition      Book  Chapter  \
0      Orthodox Jewish Bible  Bereshis        1   
1      Orthodox Jewish Bible  Bereshis        1   
2      Orthodox Jewish Bible  Bereshis        1   
3      Orthodox Jewish Bible  Bereshis        1   
4      Orthodox Jewish Bible  Bereshis        1   
...                      ...       ...      ...   
31019  Orthodox Jewish Bible  Hisgalus       22   
31020  Orthodox Jewish Bible  Hisgalus       22   
31021  Orthodox Jewish Bible  Hisgalus       22   
31022  Orthodox Jewish Bible  Hisgalus       22   
31023  Orthodox Jewish Bible  Hisgalus       22   

                                                   Verse  
0      1 In the beginning Elohim bara hashomayim (the...  
1      2 And the earth was tohu vavohu (without form,...  
2      3 And Elohim said, Let there be light: and the...  
3      4 And Elohim saw the light, that it was tov (g...  
4      5 And Elohim called the light Yom (Day), and t...  
...        

In [ ]:
#rename jewish books to english

In [30]:
temp_df=pd.read_csv("NEW_csv_folders/CONCAT_Orthodox Jewish Bible.csv")#Orthodox Jewish Bible
temp_df

,Edition,Book,Chapter,Verse
0,Orthodox Jewish Bible,Bereshis,1,1 In the beginning Elohim bara hashomayim (the...
1,Orthodox Jewish Bible,Bereshis,1,"2 And the earth was tohu vavohu (without form,..."
2,Orthodox Jewish Bible,Bereshis,1,"3 And Elohim said, Let there be light: and the..."
3,Orthodox Jewish Bible,Bereshis,1,"4 And Elohim saw the light, that it was tov (g..."
4,Orthodox Jewish Bible,Bereshis,1,"5 And Elohim called the light Yom (Day), and t..."
...,...,...,...,...
31019,Orthodox Jewish Bible,Hisgalus,22,"17 And the Ruach Hakodesh and the Kallah say, ..."
31020,Orthodox Jewish Bible,Hisgalus,22,18 I give solemn edut to everyone hearing the ...
31021,Orthodox Jewish Bible,Hisgalus,22,19 And if anyone takes away from the devarim o...
31022,Orthodox Jewish Bible,Hisgalus,22,20 He who gives solemn edut to these things sa...


In [15]:
temp_df2=pd.read_csv("NEW_csv_folders/CONCAT_Young's Literal Translation.csv")
temp_df2

,Edition,Book,Chapter,Verse,Proper nouns,Sentiment,Subjectivity,Summary
0,Young's Literal Translation,Genesis,1,1 In the beginning of God's preparing the heav...,God,0.000000,0.000000,beginning;earth
1,Young's Literal Translation,Genesis,1,"2 the earth hath existed waste and void, and d...",God,-0.100000,0.200000,face;earth;hath;waste;void;darkness
2,Young's Literal Translation,Genesis,1,"3 and God saith, ‘Let light be;’ and light is.",God,0.400000,0.700000,light;saith
3,Young's Literal Translation,Genesis,1,"4 And God seeth the light that [it is] good, a...",God,0.500000,0.666667,light;darkness
4,Young's Literal Translation,Genesis,1,"5 and God calleth to the light ‘Day,’ and to t...",God,0.400000,0.700000,darkness;evening;morning;day
...,...,...,...,...,...,...,...,...
31230,Young's Literal Translation,Revelation,22,"17 And the Spirit and the Bride say, Come; and...",NaN,0.325000,0.775000,let;water;life
31231,Young's Literal Translation,Revelation,22,18 ‘For I testify to every one hearing the wor...,God,0.000000,0.000000,scroll;prophecy
31232,Young's Literal Translation,Revelation,22,19 and if any one may take away from the words...,God,0.000000,0.000000,scroll;prophecy;part;life;city
31233,Young's Literal Translation,Revelation,22,20 he saith — who is testifying these things —...,Jesus,0.651042,0.500000,NaN


In [31]:
#j_books=[]
en_books=[]
#[j_books.append(x) for x in temp_df['Book'] if x not in j_books]
#ONLY OJB
ojb_books_right_order=['Bereshis','Shemot', 'Vayikra','Bamidbar','Devarim','Yehoshua','Shofetim','Ruth','Shmuel Alef','Shmuel Bais',
'Melachim Alef','Melachim Bais','Divrey Hayamim Alef','Divrey Hayamim Bais', 'Ezra', 'Nechemyah','Esther', 'Iyov',
'Tehillim','Mishle','Kohelet','Shir Hashirim', 'Yeshayah', 'Yirmeyah','Ekhah', 'Yechezkel','Daniel', 'Hoshea',
'Yoel','Amos','Ovadyah', 'Yonah','Michoh','Nachum','Chabakuk', 'Tzefanyah', 'Chaggai', 'Zecharyah','Malachi'
, 'Mattityahu','Markos','Lukas','Yochanan', 'Gevurot Meyruach Hakodesh', 'Romans', '1 Corinthians', '2 Corinthians',
'Galatians', 'Ephesians','Philippians', 'Colossians', '1 Thessalonians', '2 Thessalonians', '1 Timotiyos',
'2 Timotiyos', 'Titos', 'Philemon', 'Hebrews',"Ya'Akov", "1 Kefa", "2 Kefa", "1 Yochanan","2 Yochanan",
"3 Yochanan", 'Yehuda','Hisgalus']
[en_books.append(x) for x in temp_df2['Book'] if x not in en_books]
dict_book={v:k for k,v in zip(j_books,ojb_books_right_order)}
dict_book

{'Bereshis': 'Genesis',
 'Shemot': 'Exodus',
 'Vayikra': 'Leviticus',
 'Bamidbar': 'Numbers',
 'Devarim': 'Deuteronomy',
 'Yehoshua': 'Joshua',
 'Shofetim': 'Judges',
 'Ruth': 'Ruth',
 'Shmuel Alef': '1 Samuel',
 'Shmuel Bais': '2 Samuel',
 'Melachim Alef': '1 Kings',
 'Melachim Bais': '2 Kings',
 'Divrey Hayamim Alef': '1 Chronicles',
 'Divrey Hayamim Bais': '2 Chronicles',
 'Ezra': 'Ezra',
 'Nechemyah': 'Nehemiah',
 'Esther': 'Esther',
 'Iyov': 'Job',
 'Tehillim': 'Psalms',
 'Mishle': 'Proverbs',
 'Kohelet': 'Ecclesiastes',
 'Shir Hashirim': 'Song Of Songs',
 'Yeshayah': 'Isaiah',
 'Yirmeyah': 'Jeremiah',
 'Ekhah': 'Lamentations',
 'Yechezkel': 'Ezekiel',
 'Daniel': 'Daniel',
 'Hoshea': 'Hosea',
 'Yoel': 'Joel',
 'Amos': 'Amos',
 'Ovadyah': 'Obadiah',
 'Yonah': 'Jonah',
 'Michoh': 'Micah',
 'Nachum': 'Nahum',
 'Chabakuk': 'Habukkuk',
 'Tzefanyah': 'Zephaniah',
 'Chaggai': 'Haggai',
 'Zecharyah': 'Zechariah',
 'Malachi': 'Malachi',
 'Mattityahu': 'Matthew',
 'Markos': 'Mark',
 'Lukas'

In [32]:
temp_df['Book']=[dict_book[x] for x in temp_df['Book']]
temp_df

,Edition,Book,Chapter,Verse
0,Orthodox Jewish Bible,Genesis,1,1 In the beginning Elohim bara hashomayim (the...
1,Orthodox Jewish Bible,Genesis,1,"2 And the earth was tohu vavohu (without form,..."
2,Orthodox Jewish Bible,Genesis,1,"3 And Elohim said, Let there be light: and the..."
3,Orthodox Jewish Bible,Genesis,1,"4 And Elohim saw the light, that it was tov (g..."
4,Orthodox Jewish Bible,Genesis,1,"5 And Elohim called the light Yom (Day), and t..."
...,...,...,...,...
31019,Orthodox Jewish Bible,Revelation,22,"17 And the Ruach Hakodesh and the Kallah say, ..."
31020,Orthodox Jewish Bible,Revelation,22,18 I give solemn edut to everyone hearing the ...
31021,Orthodox Jewish Bible,Revelation,22,19 And if anyone takes away from the devarim o...
31022,Orthodox Jewish Bible,Revelation,22,20 He who gives solemn edut to these things sa...


In [176]:
temp_df

,Edition,Book,Chapter,Verse
0,New Revised Standard Version,Genesis,1,1 In the beginning when God created the heaven...
1,New Revised Standard Version,Genesis,1,2 the earth was a formless void and darkness c...
2,New Revised Standard Version,Genesis,1,"3 Then God said, “Let there be light”; and the..."
3,New Revised Standard Version,Genesis,1,4 And God saw that the light was good; and God...
4,New Revised Standard Version,Genesis,1,"5 God called the light Day, and the darkness h..."
...,...,...,...,...
31148,New Revised Standard Version,Revelation,22,"17 The Spirit and the bride say, “Come.”And le..."
31149,New Revised Standard Version,Revelation,22,18 I warn everyone who hears the words of the ...
31150,New Revised Standard Version,Revelation,22,19 if anyone takes away from the words of the ...
31151,New Revised Standard Version,Revelation,22,"20 The one who testifies to these things says,..."


In [33]:
temp_df.to_csv('NEW_csv_folders/CONCAT_Orthodox Jewish Bible.csv', index=False,header=True)

In [118]:
#check book names
df_l= [pd.read_csv('NEW_csv_folders/'+x) for x in os.listdir('NEW_csv_folders/') if x.endswith('.csv')]

    

In [129]:
books_l=[]
for each in df_l:
    temp_dl=[]
    print(each['Edition'][0])
    [temp_dl.append(x) for x in each['Book'] if x not in temp_dl]
    print(len(temp_dl))
    books_l.append(temp_dl)

Revised Version With Apocrypha
81
Tree Of Life Version
66
Orthodox Jewish Bible
66
Young'S Literal Translation
66
Catholic Public Domain Version
73
New World Translation
66


In [141]:
[x for x in books_l[1] if x not in books_l[2]]

['Habakkuk', 'Jacob (James)', 'Judah (Jude)', 'The Revelation']

In [140]:
for i,each in enumerate(books_l):
    print(books_l)
    print([str(j+1)+'-'+x for j,x in enumerate(books_l[i])])

['1-Genesis', '2-Exodus', '3-Leviticus', '4-Numbers', '5-Deuteronomy', '6-Joshua', '7-Judges', '8-Ruth', '9-1 Samuel', '10-2 Samuel', '11-1 Kings', '12-2 Kings', '13-1 Chronicles', '14-2 Chronicles', '15-Ezra', '16-Nehemiah', '17-Esther', '18-Job', '19-Psalms', '20-Proverbs', '21-Ecclesiastes', '22-Song Of Songs', '23-Isaiah', '24-Jeremiah', '25-Lamentations', '26-Ezekiel', '27-Daniel', '28-Hosea', '29-Joel', '30-Amos', '31-Obadiah', '32-Jonah', '33-Micah', '34-Nahum', '35-Habakkuk', '36-Zephaniah', '37-Haggai', '38-Zechariah', '39-Malachi', '40-1 Esdras', '41-2 Esdras', '42-Tobit', '43-Judith', '44-The Rest Of Esther', '45-Wisdom', '46-Ecclesiasticus', '47-Baruch', '48-Epistle Of Jeremy', '49-Song of Three', '50-Susanna', '51-Bel And The Dragon', '52-Prayer Of Manasses', '53-1 Maccabees', '54-2 Maccabees', '55-S. Matthew', '56-S. Mark', '57-S. Luke', '58-S. John', '59-The Acts', '60-Romans', '61-1 Corinthians', '62-2 Corinthians', '63-Galatians', '64-Ephesians', '65-Philippians', '66-